<a href="https://colab.research.google.com/github/SytzeAndr/NGCF_RP32/blob/master/NGCF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##The steps to take
As a rough outline, we can sketch the steps to be taken to train a Neural Graph Collaborative Filtering system as follows.

1. Parse the data in $(u, i, j)$ tuples, where we have user $u$, positive item $i$ and negative item $j$.
1. Implement the NGCF model
  1. Create a user-item interaction graph for each batch.
  1. Create a NGCF Layer that can be used to perform a MessagePassing step.
  1. Use this NGCF Layer in a Neural Net to obtain predictions for the batch.
1. Train all the parameters
  1. Construct the matrices W1_l, W2_l and E_l for each step l, initialize them with the Xavier Initializer.
  1. Sample a batch from the training data. Apply L steps of message passing, and save all the matrices W1_{1..L}, W2_{1..L}, E_{1..L}, repeat until all training data are sampled once.
  1. Construct the final embeddings for each user and item, by concatenating the embeddings for a user or item from each step, so **e_u** = **e_u^1** || ... || **e_u^L**
  1. Using the final embeddings, compute the prediction for each (user, item) pair, and compute the BPR loss.
  1. Use the BPR loss to update the matrices W1 and W2, repeat until $T$ epochs are done.
1. Generate the test output, compute recall and ndgc, compare them to the Table 3
  1. Use the trained matrices W1 and W2 to find the prediction values for each (user, item) pair in the test data.
  1. Compute the recall and normalized discounted cumulative gain (ndgc)

Since we have to compare results for step l, and the final embeddings are calculated by concatenation over the steps, we can simply perform all L steps (4 in this case), and then do the analysis over subsets of the W1 and W2 matrices.

Hyperparameters to consider:
 - Negative slope of NGCFLayer#LeakyReLU
 - Learning rate for the Adam optimizer
 - $\lambda$ in loss computation


##File loading

Here we use the Google Drive mountpoint to load files. For this to work, note the following:



*   The first time you execute this, it will provide a link, which you need to follow and give permission for Colab to access your Google Drive.
*   Make sure that the data is located in the folder `RP_data` which should be located in the root of your Drive.

In [1]:
from google.colab import drive

drive.mount('/content/drive')
data_path = './drive/My Drive/RP_data/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Loading pytorch with torch_geometric (PyG)
Some of its steps are described in this blog post:
https://towardsdatascience.com/hands-on-graph-neural-networks-with-pytorch-pytorch-geometric-359487e221a8

There is also a google colab which trains a GCN to identify 'spammers'
https://colab.research.google.com/github/zaidalyafeai/Notebooks/blob/master/Deep_GCN_Spam.ipynb#scrollTo=_4_eVOI2M4Uo

**Importing torch_geometric 1.3.2**

`torch_geometric` is a geometric deep learning extension library for PyTorch. We don't use the latest version (loaded by default by google colab) due to inconsistencies with PyTorch. This also means we have to downgrade a few other packages aswell.

In [3]:
# !pip install torch===1.2.0 torchvision===1.4.0 -f https://download.pytorch.org/whl/torch_stable.html
# these were the corresponding versions for torch-geometric 1.3.2, released 4 oct 2019, which runs on torch 1.2.0.
# grab some coffee, this might take a while
!pip install torch-scatter==1.3.1
!pip install torch-sparse==0.4.0
!pip install torch-cluster==1.4.4
!pip install torch-spline-conv==1.1.0
!pip install torch-geometric==1.3.2

  Using cached https://files.pythonhosted.org/packages/bd/5f/01c5799cd1f81f9956f03a0e1d9a861e020a598dd411d9bd3c3c1dd5b8a4/torch_cluster-1.4.4.tar.gz
  Created wheel for torch-cluster: filename=torch_cluster-1.4.4-cp36-cp36m-linux_x86_64.whl size=20361529 sha256=aa27694ef80b89130e1d4c5c2efcad00346a610a5d2b2c0f1a1c51b409b191b0
  Stored in directory: /root/.cache/pip/wheels/20/7b/ab/b3e266920055d1e51988f93a99ef8df62e399b234c8d50527f
Successfully built torch-cluster


In [4]:
import torch
import torch_cluster
import torch_geometric

# we use torch version 1.2.0 instead of the latest due to dependency errors
print(torch.__version__)
# verify that torch geometric is imported, should be 1.3.2
print(torch_geometric.__version__)

1.4.0
1.3.2


##Legacy: **DataReader class**

The DataReader class is a utility class that will help loading the data, compute useful properties of the data and create sample batches.

In [0]:
import random as rd

class DataReader(object):
  def __init__(self, path='./drive/My Drive/RP_data/raw', batch_size=1024):
    self.path = path
    self.batch_size = batch_size

    train_file = path + '/train.txt'
    test_file = path + '/test.txt'

    self.n_train = 0
    self.n_test = 0
    self.exist_users = []
    self.train_items = {}
    self.test_items = {}

    with open(train_file) as f:
      for l in f.readlines():
        if len(l) > 0:
          l = l.strip('\n')
          items = [int(i) for i in l.split(' ')]
          uid, train_items = items[0], items[1:]
          self.exist_users.append(uid)
          self.train_items[uid] = train_items
          self.n_train += len(train_items)

    with open(test_file) as f:
      for l in f.readlines():
        if len(l) > 0:
          l = l.strip('\n')
          try:
            items = [int(i) for i in l.split(' ')]
            uid, test_items = items[0], items[1:]
            self.exist_users.append(uid)
            self.test_items[uid] = test_items
            self.n_test += len(test_items)
          except Exception:
            continue

    train_max_item = max([max(items) for items in list(self.train_items.values())])
    test_max_item = max([max(items) for items in list(self.test_items.values())])
    self.n_items = max(train_max_item, test_max_item)
    self.n_users = max(self.exist_users)
  

  def sample(self):
    if self.batch_size <= self.n_users:
      users = rd.sample(self.exist_users, self.batch_size)
    else:
      users = [rd.choice(self.exist_users) for _ in range(self.batch_size)]


    def sample_pos_items_for_u(u, num):
      pos_items = self.train_items[u]
      n_pos_items = len(pos_items)
      pos_batch = []
      while True:
        if len(pos_batch) == num: break
        pos_id = np.random.randint(low=0, high=n_pos_items, size=1)[0]
        pos_i_id = pos_items[pos_id]

        if pos_i_id not in pos_batch:
          pos_batch.append(pos_i_id)
      return pos_batch


    def sample_neg_items_for_u(u, num):
      neg_batch = []
      while True:
        if len(neg_batch) == num: break
        neg_id = np.random.randint(low=0, high=self.n_items, size=1)[0]
        if neg_id not in self.train_items[u] and neg_id not in neg_batch:
          neg_batch.append(neg_id)
      return neg_batch
    

    pos_items, neg_items = [], []
    for u in users:
      pos_items += sample_pos_items_for_u(u, 1)
      neg_items += sample_neg_items_for_u(u, 1)

    return users, pos_items, neg_items


In [0]:
# small test to verify whether we can load data
dataReader = DataReader()
dataReader.sample()

In [0]:
# torch_geometric.data.Data provides a number of utility functions
# these prints are a check to verify if our graph seems valid
# print(data.is_undirected())
# print(data.is_directed())
# print(data.num_edges)
# print(data.num_nodes)
# print(data.contains_isolated_nodes())
# print(data.contains_self_loops())

# Data loading
The model will be trained on batches of $(u,i,j)$, where $u$ is a user, $i$ is an item with which $u$ interacted and $j$ is an item with which $u$ did not interact. Therefore, when loading the data, we find all the possible $(u,i,j)$ tuples and store them. The `DataConstruction` class makes data handling and parsing easier, since the processed data is stored in a separate file for easy access.

In [0]:
from torch_geometric.data import InMemoryDataset, Data
from torch.utils.data import DataLoader
import numpy as np
import os

class DataConstruction(InMemoryDataset):
  def __init__(self, root, transform=None, pre_transform=None):
    super(DataConstruction, self).__init__(root, transform, pre_transform)
    self.data = torch.load(self.processed_paths[0])
    self.n_users = torch.max(self.data[:, 0]).item()
    self.n_items = torch.max(self.data[:, 1:]).item()

  @property
  def raw_file_names(self):
    return [os.path.join(self.root, 'data.txt')]
  
  @property
  def processed_file_names(self):
    return ['processed.dataset']
  
  def download(self):
    pass
  
  def process(self):
    data = []

    for file in self.raw_file_names:
      users = []
      items = []
      pos_items = {}

      with open(file) as f:
        for l in f.readlines():
          if len(l) > 0:
            l = l.strip('\n')
            items = [int(i) for i in l.split(' ')]
            uid, items = items[0], items[1:]
            users.append(uid)
            items.extend(items)
            pos_items[uid] = np.asarray(items)
      users = np.asarray(users)
      items = np.unique(items)

      for uid in users:
        pos = pos_items[uid]
        neg = np.setdiff1d(items, pos)
        for p in pos:
          for n in neg:
            data.append([uid, p, n])

    torch.save(torch.tensor(data), self.processed_paths[0])

In [0]:
train_path = data_path + 'train'
train_data = DataConstruction(root=train_path)

# Model definition


For each user-item pair (u,i), we define the message from i to u as

$m_{u \leftarrow i} = \dfrac{1}{\sqrt{|N_u||N_i|}} (W_1 e_i + W_2(e_i \odot e_u))$

Where $N_u, N_i$ are the first hop neighbors of $u$ and $i$, $W_1, W_2$ are trainable weight matrices to distill useful information for propagation, and $e_i$ and $e_u$ are embeddings of the users and items. Note that in the special case where $u == i$, the second term is dropped.

The embedding for a user (or item) is updated through message aggregation as 

$e_u^{(l)} = LeakyReLU(m_{u \leftarrow u} + \sum_{i \in N_u} m_{u \leftarrow i})$

If the graph is constructed properly, this is simply a summation over all edges (when selfloops are included), followed by a LeakyReLU activiation.

In [0]:
from torch.nn import init, Embedding, LeakyReLU, Linear
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops

# define our NGCF layer
class NGCFLayer(MessagePassing):
  def __init__(self, embed_size):
    super(NGCFLayer, self).__init__(aggr='add') # Summation as aggregation
    # The W1 matrix
    self.lin1 = Linear(embed_size, embed_size, bias=False)
    init.xavier_normal_(self.lin1.weight)
    # The W2 matrix
    self.lin2 = Linear(embed_size, embed_size, bias=False)
    init.xavier_normal_(self.lin2.weight)
    self.act = LeakyReLU()

  def forward(self, x, edge_index):
    # Add self loops
    edge_index, _ = remove_self_loops(edge_index)
    edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

    # Compute normalization
    row, col = edge_index
    deg = degree(row, x.size(0), dtype=x.dtype)
    deg_inv_dqrt = deg.pow(-0.5)
    norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

    return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x, norm=norm)
  
  def message(self, x_i, x_j, norm):
    # Construct message
    message = self.lin1(x_j)
    # Only add the second term if it is not a self loop
    if x_i.data_ptr() != x_j.data_ptr():
      message += self.lin2(torch.mul(x_i, x_j))
    return message

  def update(self, aggr_out):
    # Return the LeakyReLU result
    return self.act(aggr_out)

In [0]:
## Define our NN using the NGCF layer definition

embed_size = 64
batch_size = 1024
num_epochs = 3

class Net(torch.nn.Module):
  def __init__(self, n_layers, num_embeddings):
    super(Net, self).__init__()

    self.embedding = Embedding(num_embeddings=num_embeddings, embedding_dim=embed_size)
    init.xavier_uniform_(self.embedding.weight)

    self.ngcf_layers = []
    for i in range(n_layers):
      layer = NGCFLayer(embed_size=embed_size)
      self.ngcf_layers.append(layer)
    
  def forward(self, data):
    print(data)
    x, edge_index, batch = data.x, data.edge_index, data.batch
    for layer in self.ngcf_layers:
      x = layer(x, edge_index)
    return x

# Optimization

In [0]:
# Utility to parse a batch of triplets into a Graph structure
def parse_batch(batch):
  edges = []
  # define an edge for each (u, i)
  for tuple in batch:
    # increase the item id by the number of users to distinct the nodes from users
    edges.append([tuple[0], tuple[1] + train_data.n_users])
    edges.append([tuple[1] + train_data.n_users, tuple[0]])
  
  # following example from https://pytorch-geometric.readthedocs.io/en/latest/notes/introduction.html
  edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
  
  # Needs testing
  users = batch[:, 0]
  items = batch[:, 1:] + train_data.n_users
  x = users.extend(items)
  
  return Data(x=x, edge_index=edge_index)


def optimize():
  model.train()
  loss_all = 0
  for dataset in train_loader:
    dataset = dataset.to(device)
    optimizer.zero_grad()
    output = model(dataset)
    label = dataset.y.to(device)
    loss = crit(output, label)
    loss.backward()
    loss_all += dataset.num_graphs * loss_item()
    optimizer.step()
  return loss_all / len(train_loader)

# to enable running on the GPU in google colab: 
# Runtime -> select Change runtime type -> select GPU and hit Save
# device = torch.device('cuda')
# model = Net(n_layers=1).to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
# crit = torch.nn.BCELoss()

model = Net(n_layers=1, num_embeddings=train_data.n_users + train_data.n_items)
train_loader = DataLoader(train_data.data, batch_size=batch_size, shuffle=True)

batch = parse_batch(next(iter(train_loader)))
# output = model(batch)
# print(output)

# Step 2

First we initialize the initial weights and embeddings. This is performed by [Xavier initialization](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf).

In [0]:
import math

def xavier(m, h):
  return torch.Tensor(m, h).uniform_(-1, 1)*math.sqrt(6./(m+h))

class NGCF(object):
  def __init__(self, n_users, n_items, emb_dim=64, n_layers=4):
    self.n_users = n_users
    self.n_items = n_items
    self.emb_dim = emb_dim
    self.weight_size = list(np.repeat(emb_dim, n_layers))
    self.n_layers = n_layers

  def _init_weights(self):
    all_weights = dict()
    all_weights['user_embedding'] = xavier(self.n_users, self.emb_dim) 
    all_weights['item_embedding'] = xavier(self.n_items, self.emb_dim)
    self.weight_size_list = [self.emb_dim] + self.weight_size
    for k in range(self.n_layers):
      all_weights['W_gc_%d' % k] = xavier(self.weight_size_list[k], self.weight_size_list[k+1])
      all_weights['b_gc_%d' % k] = xavier(1, self.weight_size_list[k+1])
      all_weights['W_bi_%d' % k] = xavier(self.weight_size_list[k], self.weight_size_list[k + 1])
      all_weights['b_bi_%d' % k] = xavier(1, self.weight_size_list[k + 1])
      all_weights['W_mlp_%d' % k] = xavier(self.weight_size_list[k], self.weight_size_list[k+1])
      all_weights['b_mlp_%d' % k] = xavier(1, self.weight_size_list[k+1])
    return all_weights



In [0]:
# set initial weight
# emd_dim and n_layers can be tweaked to our liking
ncfg_obj = NGCF(n_users=dataReader.n_users, n_items=dataReader.n_items, emb_dim=64, n_layers=4)

# store all our initial weights in some dict object
ncfg_weights = ncfg_obj._init_weights()

# optional prints: check the amount of user embeddings as a verification
print(ncfg_weights.keys())
print(len(ncfg_weights['user_embedding']))

dict_keys(['user_embedding', 'item_embedding', 'W_gc_0', 'b_gc_0', 'W_bi_0', 'b_bi_0', 'W_mlp_0', 'b_mlp_0', 'W_gc_1', 'b_gc_1', 'W_bi_1', 'b_bi_1', 'W_mlp_1', 'b_mlp_1', 'W_gc_2', 'b_gc_2', 'W_bi_2', 'b_bi_2', 'W_mlp_2', 'b_mlp_2', 'W_gc_3', 'b_gc_3', 'W_bi_3', 'b_bi_3', 'W_mlp_3', 'b_mlp_3'])
52642
